In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict

from sklearn.preprocessing import KBinsDiscretizer

In [87]:
# https://onti.ai-academy.ru/competition

In [ ]:
train = pd.read_csv('data/transactions_train.csv')

In [14]:
train_labels = pd.read_csv('data/train_target.csv')

In [3]:
train.head(5)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341


In [5]:
encoders = {}

for group in train.small_group.unique():
    encoders[group] = KBinsDiscretizer(encode='ordinal')
    condition = (train.small_group == group)
    values = train.loc[condition, 'amount_rur'].values.reshape(-1, 1)
    train.loc[condition, 'amount_bin'] = encoders[group].fit_transform(values)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:159: UserWarning: Feature 0 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:159: UserWarning: Feature 0 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:193: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_discretization.py:159: UserWarning: Feature 0 is constant and will be replaced wit

In [6]:
train['trans_id'] = 'bin' + train['amount_bin'].astype(int).astype(str) + '_trans' + train['small_group'].astype(str)

In [7]:
train.head(5)

,client_id,trans_date,small_group,amount_rur,amount_bin,trans_id
0,33172,6,4,71.463,4.0,bin4_trans4
1,33172,6,35,45.017,3.0,bin3_trans35
2,33172,8,11,13.887,2.0,bin2_trans11
3,33172,9,11,15.983,2.0,bin2_trans11
4,33172,10,11,21.341,3.0,bin3_trans11


In [9]:
train_data = dict()
trans_encoder = defaultdict(set)

for idx, row in tqdm(train.iterrows()):
    client_id = row['client_id']
    if client_id not in train_data:
        train_data[client_id] = defaultdict(list)
    
    trans_date = row['trans_date']
    if trans_date not in trans_encoder[client_id]:
        trans_encoder[client_id].add(trans_date)

    trans_date_encoded = len(trans_encoder[client_id]) - 1
    train_data[client_id][trans_date_encoded].append(row['trans_id'])

26450577it [2:00:27, 3659.69it/s]


In [10]:
import pickle
pickle.dump(train_data, open('data/train_data_transactions.pkl', 'wb'))

In [11]:
len(train_data)

30000

In [26]:
client2bin = dict(train_labels.values)

In [29]:
train_df = []
num_days = 5

for client_id, transactions in tqdm(train_data.items()):
    examples = defaultdict(list)
    for day, ids in transactions.items():
        day_id = int(day / num_days)
        examples[day_id].extend(ids)
    
    for _, ex in examples.items():
        train_df.append([client_id, ' '.join(ex), client2bin[client_id]])
        

100%|██████████| 30000/30000 [00:21<00:00, 1425.15it/s]


In [30]:
final_dataset = pd.DataFrame(train_df, columns=['client_id', 'transactions', 'label'])

In [37]:
final_dataset[['transactions', 'label']].to_csv('final_dataset.csv', index=False)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
tr, te = train_test_split(final_dataset[['transactions', 'label']], random_state=24, test_size=0.05)

In [41]:
tr.to_csv('data_transactions/train.csv', index=False)
te.to_csv('data_transactions/test.csv', index=False)

In [86]:
# enc = KBinsDiscretizer(encode='ordinal', strategy='uniform')

# enc.fit_transform(np.arange(50).reshape(-1, 1))